In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Dropout,
    Flatten,
    Dense,
    Reshape,
    GlobalAveragePooling2D,
)

In [ ]:
import os
import numpy as np
import cv2
from concurrent.futures import ThreadPoolExecutor


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images


def resize_and_normalize_image(img, target_size):
    # Maintain aspect ratio and add padding
    old_size = img.shape[:2]  # old_size is in (height, width) format
    ratio = float(target_size[0]) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])

    resized_img = cv2.resize(
        img, (new_size[1], new_size[0])
    )  # resize() requires (width, height)

    delta_w = target_size[1] - new_size[1]
    delta_h = target_size[0] - new_size[0]

    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]
    new_img = cv2.copyMakeBorder(
        resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color
    )

    normalized_img = new_img / 255.0  # Normalize pixel values
    return normalized_img


def resize_and_normalize_images(images, target_size):
    with ThreadPoolExecutor() as executor:
        resized_images = list(
            executor.map(
                lambda img: resize_and_normalize_image(img, target_size), images
            )
        )
    return np.array(resized_images)


train_folder = (
    r"C:\Users\kunjf\Documents\AI Saksham\Image Compression\data2\train_images"
)
test_folder = r"C:\Users\kunjf\Documents\AI Saksham\Image Compression\data2\test_images"
save_path = r"C:\Users\kunjf\Documents\AI Saksham\Image Compression\data2"
target_size = (128, 128)  # Set your desired target size for images

train_images = load_images_from_folder(train_folder)
test_images = load_images_from_folder(test_folder)

X_train = resize_and_normalize_images(train_images, target_size)
X_test = resize_and_normalize_images(test_images, target_size)

np.save(os.path.join(save_path, "X_train2.npy"), X_train)
np.save(os.path.join(save_path, "X_test2.npy"), X_test)

In [5]:
def build_flexible_autoencoder():
    input_img = Input(
        shape=input_shape
    )  # Input shape determined by your maximum image size


# Define input shape
input_shape = (None, None, 3)  # Dynamic input shape


def build_flexible_autoencoder():
    input_img = Input(shape=(None, None, 3))

    # Encoder
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(input_img)
    x = MaxPooling2D((2, 2), padding="same")(x)
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    encoded = MaxPooling2D((2, 2), padding="same")(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

In [ ]:
autoencoder = build_flexible_autoencoder()
autoencoder.compile(optimizer="adam", loss="mean_squared_error")
autoencoder.summary()

In [3]:
import numpy as np

# Load the data from the saved files
X_train = np.load(
    r"C:\Kunj\Programming\AI Saksham\Image Compression\data2\X_train_64.npy"
)
X_test = np.load(
    r"C:\Kunj\Programming\AI Saksham\Image Compression\data2\X_test_64.npy"
)

In [ ]:
autoencoder.fit(
    X_train, X_train, epochs=15, batch_size=8, validation_data=(X_test, X_test)
)

In [ ]:
autoencoder.save("color_128.h5")

In [ ]:
from keras.models import load_model

model_color = load_model("color_128.h5")

In [ ]:
# Predict and display results
pred = model_color.predict(X_test)

In [6]:
from PIL import Image

In [ ]:
# Load your color image
image_path = "data2/train_images/image_89.png"
image = Image.open(image_path)
print("Original Image Shape:", image.size)

# Ensure the image is in RGB format
image = image.convert("RGB")  # Resize and convert to RGB
image = np.array(image) / 255.0
print("Processed Image Shape:", image.shape)
print("Model Input Shape:", model_color.input_shape)

# Reshape and predict
input_image = np.expand_dims(image, axis=0)
print("Input Image Shape for Prediction:", input_image.shape)

predicted_image = model_color.predict(input_image)
print("Predicted Image Shape:", predicted_image.shape)

# Display original and compressed images
plt.figure(figsize=(10, 4))

# Display original image
ax = plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")

# Display compressed image
ax = plt.subplot(1, 2, 2)
plt.imshow(predicted_image[0])
plt.title("Compressed Image")
plt.axis("off")

plt.show()

In [ ]:
# Example usage:
image_path = "kunj.png"
image = Image.open(image_path)
print("Original Image Shape:", image.size)

# Ensure the image is in RGB format
image = image.convert("RGB")  # Resize and convert to RGB
image = np.array(image) / 255.0
# Get the dimensions of the image
height, width, channels = image.shape

print("Processed Image Shape:", image.shape)
print("Model Input Shape:", model_color.input_shape)

# Reshape and predict
input_image = np.expand_dims(resized_image, axis=0)
print("Input Image Shape for Prediction:", input_image.shape)

predicted_image = model_color.predict(input_image)
print("Predicted Image Shape:", predicted_image.shape)


# Display original image
ax = plt.subplot(1, 2, 1)
plt.imshow(resized_image)
plt.title("Original Image")
plt.axis("off")

# Display compressed image
ax = plt.subplot(1, 2, 2)
plt.imshow(predicted_image[0])
plt.title("Compressed Image")
plt.axis("off")

plt.show()

In [ ]:
# Scale the pixel values back to the original range (0-255) for uint8 format
compressed_img_uint8 = (predicted_image[0] * 255).astype(np.uint8)

# Save the compressed image
cv2.imwrite(
    "compressed_image_128.jpg",
    cv2.cvtColor(compressed_img_uint8, cv2.COLOR_RGB2BGR),
)